<a href="https://colab.research.google.com/github/icculp/route-66-scrape_public/blob/master/places_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import requests

In [19]:
columns = ['RoadTripStopTypeID',
'CategoryID',
'RoadTripStopName',
'RoadTripStopDescription',
'RoadTripStopAddressLine1',
'RoadTripStopCity',
'RoadTripStopState',
'RoadTripStopZip',
'RoadTripStopLat',
'RoadTripStopLong',
'RoadTripStopEmail',
'RoadTripStopPhone',
'TicketBookingURL',
'Image',
'EventDateTime',
'RoadTripStopURL',
'PlaceID',
'SunOpen',
'SunClose',
'MonOpen',
'MonClose',
'TueOpen',
'TueClose',
'WedOpen',
'WedClose', 
'ThurOpen',
'ThurClose',
'FriOpen',
'FriClose',
'SatOpen',
'SatClose']

You can either copy/paste the dictionary output from the script historic66 outputs, or load the data in from file. In colab you can upload to session storage

In [20]:
# places = {}
# print(len(places))

In [21]:
!ls

historic66.json  sample_data


In [22]:
filename = 'historic66.json'
with open(filename, 'r', encoding='utf-8') as f:
    places = json.load(f)
print(len(places))
# dict(list(places.items())[0:5])  # preview places

152


In [23]:
key = ''  # leaving key out of repo for 

In [24]:
Places_final = dict()
periods_default = dict()
i = 0
no_candidates = []
for Place, details in places.items():
    address = details['address']
    Description = details['description']
    Image = details['image']
    print(Place, end='')
    append = Place.replace(' ', '%20') + '%20' + address.replace(' ', '%20').replace('\n', '%20')
    pid = requests.get('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=' + append + '&inputtype=textquery&key=' + key)
    try:
        Place_id = pid.json()['candidates'][0].get('place_id')
    except:
        no_candidates.append([Place, address, Description, Image])
        print("\nNo candidates for [{}], [{}], skipping".format(Place, address.replace('\n', ' ')))
        i += 1
        print(i, end='..')
        continue
    p =  requests.get('https://maps.googleapis.com/maps/api/place/details/json?place_id=' + Place_id + '&key=' + key)
    result = p.json()['result']

    adr_address = result.get('adr_address', 'NULL')
    soup = BeautifulSoup(adr_address, features="html.parser")
    Address = soup.find("span", {"class": "street-address"}).text if soup.find("span", {"class": "street-address"}) is not None else "NULL"
    City = soup.find("span", {"class": "locality"}).text if soup.find("span", {"class": "locality"}) is not None else "NULL"
    State = soup.find("span", {"class": "region"}).text if soup.find("span", {"class": "region"}) is not None else "NULL"
    Zip = soup.find("span", {"postal-code"}).text if soup.find("span", {"class": "postal-code"}) is not None else "NULL"

    Lat = str(result.get('geometry', 'NULL').get('location', 'NULL').get('lat', 'NULL'))
    Long = str(result.get('geometry', 'NULL').get('location', 'NULL').get('lng', 'NULL'))
    Email = 'NULL'
    Phone = result.get('formatted_phone_number', 'NULL')
    BookingUrl = result.get('website', 'NULL')
    URL = result.get('url', 'NULL')

    Periods = result.get('opening_hours')
    days = {j: {'open': 'NULL', 'close': 'NULL'} for j in range(7)}
    if Periods is not None:
        for day in Periods['periods']: 
            for k, v in day.items():
                timeForm = v.get('time')
                timeForm = timeForm[:2] + ':' + timeForm[2:]
                days[v['day']].update({k: timeForm})
    hoo = []  # hours of operation
    for k, d in days.items():
        hoo.append(d.get('open'))
        hoo.append(d.get('close'))

    line_items = [str(2), 'NULL', Place, Description, Address, City, State, Zip, Lat, Long, Email, Phone, BookingUrl, Image, 'NULL', URL, Place_id]
    line_items += hoo  # [SunOpen, SunClose, MonOpen, MonClose, TueOpen, TueClose, WedOpen, WedClose, ThurOpen, ThurClose, FriOpen, FriClose, SatOpen, SatClose]
    Places_final.update({Place_id: line_items})

    i += 1
    #if i == 5:
    #    break
    print(i, end='..')

Allen’s Conoco Fillin’ Station1..Mickey Mantle’s Childhood Home2..Mickey Mantle Field3..Picher – Ghost Town4..Constable Campbell Memorial5..The Hitch n Post Flea Market6..Dairy King7..Coleman Theater8..Miami Convention and Visitors Bureau9..Route 66 Vintage Iron10..Rte 66 Historic Marker11..Nowhere on Route 6612..Rest Haven Sign13..Afton Station14..Avon Court15..Vinita Bridge16..Clanton’s Cafe17..The Coffee Shelter18..Old Service Station19..Route 66 Inn20..Diamond Rio
No candidates for [Diamond Rio], [437950 US-60 Vinita, OK 74301], skipping
21..Hi-Way Café22..Pryor Creek Bridge23..Pedestrian Underpass24..Ed Galloway’s Totem Pole Park25..Bunion Derby Historic Marker26..1920s Texaco station27..Claremore Motor Inn28..J.M. Davis Museum29..Will Rogers Museum30..Ron’s Hamburgers & Chili31..Correll Museum32..Catoosa Inn & Suites33..Tulsa Sign Route 6634..Tulsa Fire Station 66 “Keepers of the Mother Road”35..The Nut House36..Twin Bridges Verdigris
No candidates for [Twin Bridges Verdigris], [

In [25]:
print("No candidates found for the following items, totalling {} places".format(len(no_candidates)))
no_candidates

No candidates found for the following items, totalling 9 places


[['Diamond Rio',
  '437950 US-60\nVinita, OK 74301',
  'NULL',
  'https://www.historic66.com/wp-content/uploads/2020/12/340de1d2aba19238f1c8b9d64475037a_.jpeg'],
 ['Twin Bridges Verdigris',
  'Verdigris, OK 74019',
  'NULL',
  'https://www.historic66.com/wp-content/uploads/2020/12/906e807e999a0f93f9bc47020ad0dd1c_.jpeg'],
 ['Old Bridge',
  'Old Route 66\nBristow, OK 74010',
  'The old bridge was removed from the site in 2018 year.',
  'https://www.historic66.com/templates/data/images/sharepoi.svg'],
 ['Stroud Trading Building',
  '201 W Main St\nStroud, OK 74079',
  'Constructed in 1901, this is a two-story red brick building with sandstone trim. At the peak of the front facade the name Stroud Trading Company is built into the wall. At its building, owners boasted that it was the longest floor space of any commercial structure in the Oklahoma Territory. The original owners, the Stroud Trading Company dedicated the second floor of the building as a lodge and operahouse in 1902. This was

In [26]:
# dict(list(Places_final.items())[0:5])  # preview places

In [27]:
Lines_final = list()

for plc_id, plc_line in Places_final.items():
    '''line_string = ''
    for item in range(len(plc_line)):
        line_string += str(plc_line[item])
        if item != len(plc_line) - 1:
            line_string += '|'''
    Lines_final.append(plc_line)
# Lines_final

In [28]:
df_final = pd.DataFrame(Lines_final, columns=columns)
print(len(df_final))
df_final

140


,RoadTripStopTypeID,CategoryID,RoadTripStopName,RoadTripStopDescription,RoadTripStopAddressLine1,RoadTripStopCity,RoadTripStopState,RoadTripStopZip,RoadTripStopLat,RoadTripStopLong,RoadTripStopEmail,RoadTripStopPhone,TicketBookingURL,Image,EventDateTime,RoadTripStopURL,PlaceID,SunOpen,SunClose,MonOpen,MonClose,TueOpen,TueClose,WedOpen,WedClose,ThurOpen,ThurClose,FriOpen,FriClose,SatOpen,SatClose
0,2,NULL,Allen’s Conoco Fillin’ Station,Allen’s Conoco Fillin’ Station in Commerce was...,101 S Main St,Commerce,OK,74339-2217,36.9329867,-94.8773057,NULL,NULL,NULL,https://www.historic66.com/wp-content/uploads/...,NULL,https://maps.google.com/?cid=4232581255271158399,ChIJ0SZ4OkkXyIcRfxJq-zkmvTo,00:00,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,2,NULL,Mickey Mantle’s Childhood Home,Private residence,319 S Quincy St,Commerce,OK,74339,36.9301322,-94.8750348,NULL,NULL,http://www.atlasobscura.com/places/home-of-mic...,https://www.historic66.com/templates/data/imag...,NULL,https://maps.google.com/?cid=10596678301593638739,ChIJ68REe0gXyIcRUxeUjcX2DpM,00:00,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,Mickey Mantle Field,"In 2010, Commerce officials dedicated a statue...",400 S Mickey Mantle Blvd,Commerce,OK,74339,36.9254514,-94.8726975,NULL,NULL,http://mickeymantleclassic.com/,https://www.historic66.com/wp-content/uploads/...,NULL,https://maps.google.com/?cid=3170354839951260108,ChIJsxfepjgXyIcRzDk8_-1c_ys,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,2,NULL,Picher – Ghost Town,"In 1913, as the Tri-State district expanded, l...",NULL,Picher,OK,74360,36.9870116,-94.8307844,NULL,NULL,NULL,https://www.historic66.com/wp-content/uploads/...,NULL,"https://maps.google.com/?q=Picher,+OK+74360,+U...",ChIJ5dEhIAsRyIcRmlKi8ZIsd9s,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,2,NULL,Constable Campbell Memorial,Constable William Campbell was shot and killed...,618 Commerce Ave,Commerce,OK,74339-2402,36.9330746,-94.8714607,NULL,NULL,NULL,https://www.historic66.com/templates/data/imag...,NULL,"https://maps.google.com/?q=618+Commerce+Ave,+C...",Eik2MTggQ29tbWVyY2UgQXZlLCBDb21tZXJjZSwgT0sgNz...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2,NULL,Elm Motel Sign,NULL,Elm Ave,Erick,OK,73645,35.2098199,-99.87173849999999,NULL,NULL,NULL,https://www.historic66.com/wp-content/uploads/...,NULL,"https://maps.google.com/?q=Elm+Ave,+Erick,+OK+...",Eh1FbG0gQXZlLCBFcmljaywgT0sgNzM2NDUsIFVTQSIuKi...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
136,2,NULL,Sandhill Curiosity Shop,"Located in downtown Erick, this unusual shop o...",201 S Sheb Wooley St,Erick,OK,73645,35.2137429,-99.8668143,NULL,(580) 729-1747,http://sandhill-shop.edan.io/,https://www.historic66.com/wp-content/uploads/...,NULL,https://maps.google.com/?cid=5640198007820841505,ChIJT-Mo6ATLq4cRIfIGfzQCRk4,NULL,NULL,NULL,NULL,11:00,17:30,11:00,17:30,11:00,17:30,11:00,17:30,10:00,16:00
137,2,NULL,West Winds Motel,This motel was built in 1948 and added to the ...,623 Roger Miller Blvd,Erick,OK,73645,35.21514519999999,-99.8595938,NULL,NULL,NULL,https://www.historic66.com/wp-content/uploads/...,NULL,https://maps.google.com/?cid=11694435321133233594,ChIJdelMihnLq4cRunXt_Pb6SqI,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
138,2,NULL,One Room Jail,Built in 1910 the single room jail features an...,NULL,Texola,OK,73668,35.2200789,-99.9887114,NULL,NULL,NULL,https://www.historic66.com/wp-content/uploads/...,NULL,https://maps.google.com/?cid=18143444693527955673,ChIJDWUZph80qocR2QiuF9J2yvs,00:00,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [29]:
fileName = 'places_final.txt'
df_final.to_csv(fileName, sep="|", quoting=csv.QUOTE_NONNUMERIC, index=False)

In [30]:
!ls

historic66.json  places_final.txt  sample_data
